In [1]:
import warnings
warnings.filterwarnings('ignore')
import pickle
import numpy as np
import pandas as pd
import json
from textblob import TextBlob
import nltk
from scipy import spatial
import torch
import spacy
en_nlp = spacy.load('en_core_web_sm')

## Convert Json to Pandas Dataframe

In [2]:
#Load the training Dataset
train = pd.read_csv("trainingDataset.csv")

In [3]:
train.shape

(1605, 6)

In [4]:
train.head(3)

,date,title,category,link,abstract,paragraphs
0,21.04.2020,Barack Obama,NaN,https://en.wikipedia.org/wiki/Barack_Obama,Barack Hussein Obama II ( (listen); born Augus...,['Barack Hussein Obama II ( (listen); born Aug...
1,21.04.2020,Joe Biden,NaN,https://en.wikipedia.org/wiki/Joe_Biden,Joseph Robinette Biden Jr. (; born November 20...,['Joseph Robinette Biden Jr. (; born November ...
2,21.04.2020,George W. Bush,NaN,https://en.wikipedia.org/wiki/George_W._Bush,"George Walker Bush (born July 6, 1946) is an A...","['George Walker Bush (born July 6, 1946) is an..."


In [5]:
df = pd.read_csv("trainingDataset.csv")

## Create dictionary of sentence embeddings for faster computation

In [6]:
paras = list(df["abstract"].drop_duplicates().reset_index(drop= True))

In [7]:
len(paras)

1568

In [8]:
#Join using TextBlob
blob = TextBlob(" ".join(paras))
sentences = [item.raw for item in blob.sentences]

In [9]:
len(sentences)

1559

In [10]:
#Load the InferSent Sentence Embeddings V1 and V2
from models import InferSent
V = 2
MODEL_PATH = 'C:\\Users\\Shilpu\\NLPProject\\QuestionAnsweringSystem\\InferSent\\encoder\\infersent%s.pkl' % V
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': V}
infersent = InferSent(params_model)
infersent.load_state_dict(torch.load(MODEL_PATH))

<All keys matched successfully>

In [11]:
#Load the GloVe embedding
W2V_PATH = 'InferSent/dataset/GloVe/glove.840B.300d.txt'
infersent.set_w2v_path(W2V_PATH)

In [12]:
#Build the vocabulary.
infersent.build_vocab(sentences, tokenize=True)

Found 6141(/7255) words with w2v vectors
Vocab size : 6141


In [13]:
#Build the dictionary embeddings
dict_embeddings = {}
for i in range(len(sentences)):
    print(sentences[i])
    dict_embeddings[sentences[i]] = infersent.encode([sentences[i]], tokenize=True)
    print( dict_embeddings[sentences[i]])

Barack Hussein Obama II ( (listen); born August 4, 1961) is an American politician and attorney who served as the 44th president of the United States from 2009 to 2017.
[[-0.02874381  0.04943122  0.20910737 ...  0.1316887   0.04436744
   0.11170954]]
Joseph Robinette Biden Jr. (; born November 20, 1942) is an American politician who served as the 47th vice president of the United States from 2009 to 2017 and represented Delaware in the U.S. Senate from 1973 to 2009.
[[0.01917268 0.05178139 0.21973753 ... 0.06421494 0.0845053  0.11733628]]
George Walker Bush (born July 6, 1946) is an American politician and businessman who served as the 43rd president of the United States from 2001 to 2009.
[[-0.02874381  0.04943122  0.16229874 ...  0.06349707  0.054673
   0.1077443 ]]
Donald John Trump (born June 14, 1946) is the 45th and current president of the United States.
[[-0.01994209  0.06318004  0.1411632  ...  0.02324948  0.08776435
   0.09802058]]
George Washington (February 22, 1732 – Decem

[[-0.00239139  0.052432    0.17491902 ...  0.07826648  0.07221488
   0.11393397]]
Glenn Thomas Jacobs (born April 26, 1967) is an American professional wrestler, actor, author, businessman, and politician.
[[-0.00765041  0.04943122  0.15700327 ...  0.02883284  0.07050837
   0.08509059]]
Ben Lewis Jones (born August 30, 1941) is an American actor, politician, playwright and essayist, best known for his role as Cooter Davenport in The Dukes of Hazzard.
[[-0.02874381  0.06461575  0.16083872 ...  0.03310979  0.1101112
   0.09663422]]
John Augustus Kelly Jr. (September 16, 1927 – November 7, 1992), known professionally as Jack Kelly, was an American film and television actor most noted for the role of "Bart Maverick" in the television series Maverick, which ran on ABC from 1957-62.
[[-0.02874381  0.14793836  0.18929248 ...  0.09758885  0.0775933
   0.10871954]]
Sheila James Kuehl (born February 9, 1941) is an American politician, former child actress, and currently the member of the Los Ang

[[-0.02874381  0.11506773  0.16565734 ...  0.08459042  0.0848108
   0.13762784]]
Jean Lapointe,  (born December 6, 1935) is a Quebecois-Canadian actor, comedian and singer as well as a former Canadian Senator.
[[0.05079743 0.09485228 0.17493756 ... 0.01017086 0.0423076  0.09344561]]
Alexandrine Latendresse (born April 30, 1984) was the New Democratic Party Member of Parliament for Louis-Saint-Laurent and was elected in the 2011 Canadian federal election.
[[0.02062606 0.04943122 0.16068846 ... 0.09338584 0.04070921 0.11247552]]
Susan Leblanc is a Canadian politician and actress who was elected to the Nova Scotia House of Assembly in the 2017 provincial election.
[[0.00389284 0.04943122 0.13245724 ... 0.08254571 0.13907476 0.10766938]]
Jean-Louis Roux,  (May 18, 1923 – November 28, 2013) was a Canadian politician, entertainer and playwright who was briefly the 26th Lieutenant Governor of Quebec.
[[-0.0266937   0.11887632  0.17233463 ...  0.03127833  0.11604699
   0.09844093]]
Lenore Eliz

[[-0.00926997  0.07433592  0.11863941 ...  0.05436023  0.1342958
   0.09925629]]
James Keith O'Neill Edwards, DFC (23 March 1920 – 7 July 1988) was an English comedy writer and actor on radio and television, best known as Pa Glum in Take It From Here and as headmaster "Professor" James Edwards in Whack-O!.
[[-0.00795411  0.16860749  0.18895619 ...  0.11158337  0.09250305
   0.10228392]]
Andrew Matthew William Faulds (1 March 1923 – 31 May 2000) was a British actor and politician.
[[-0.02874381  0.06878513  0.15691224 ...  0.01038966  0.06424592
   0.06902032]]
Richard Franklin (15 July 1948 – 11 July 2007) was an Australian  film director.
[[-0.00497419  0.04943122  0.1425344  ...  0.04837483  0.0659315
   0.06991857]]
James Gaddas (born 9 June 1960 in Ragworth, Stockton-on-Tees, County Durham) is an English actor best known for playing Governing Governor Neil Grayling in ITV's Bad Girls.
[[-0.00427608  0.1770997   0.17344311 ...  0.09489776  0.0672026
   0.0960303 ]]
Helen Griffin (19

[[0.04530294 0.11126038 0.21186602 ... 0.06769342 0.07396746 0.10148454]]
Mimi Chakraborty (born 11 February 1989) is an Indian actress and politician.
[[-0.0063203   0.04943122  0.19727132 ...  0.0344511   0.03483311
   0.03894014]]
Gourang Chakraborty (born 16 June 1950), better known by his stage name Mithun Chakraborty (informally referred to as Mithun Da), is an Indian film actor, singer, producer, writer, social worker, entrepreneur, television presenter and a former Rajya Sabha Member of Parliament.
[[0.0058172  0.1331805  0.20949285 ... 0.08181215 0.08321298 0.09915087]]
Usashi Chakraborty is a Bengali film actress.
[[-0.00657415  0.04943122  0.23271789 ... -0.01258784 -0.04181507
   0.04470096]]
Honnasandra Narasimhaiah Chandrashekar, known popularly as Mukhyamantri Chandru (born 28 August 1953), is an Indian actor and politician.
[[0.0095729  0.12391892 0.16018623 ... 0.01713533 0.04721598 0.07645848]]
Locket Chatterjee (born 4 December 1974) is a Bengali actress, Indian poli

...

[[-0.00885168  0.1506995   0.16554983 ...  0.11112928  0.14070082
   0.10678719]]
Maurice Joseph Rioli (1 September 1957 – 25 December 2010) was an Australian rules footballer who represented St Mary's Football Club in the Northern Territory Football League (NTFL), South Fremantle in the West Australian Football League (WAFL) and Richmond in the Victorian Football League.
[[0.00717647 0.1800903  0.1456698  ... 0.06063615 0.09293817 0.06966899]]
Gregory John Rowell (born 1 September 1966 in Lindfield, New South Wales) is an Australian cricket player, who has played for the Queensland Bulls, New South Wales Blues, and Tasmanian Tigers.
[[-0.0271304   0.1053493   0.22716396 ...  0.08558145  0.03705371
   0.11568356]]
Kevin John Ryan (born 20 June 1984) is an Irish actor, known for his television roles in the TV series Copper and Crossbones.
[[-0.02874381  0.10000181  0.2356828  ...  0.1370204   0.09122743
   0.09478533]]
Sir Arthur Campbell Rymill (8 December 1907 – 27 March 1989) was a b

[[-0.00519743  0.10680354  0.12630084 ...  0.03382147  0.0739223
   0.04958379]]
Antonio Zappia (born 13 June 1952) is an Australian politician and former powerlifting champion.
[[0.01024176 0.08429751 0.15794462 ... 0.03074836 0.0394147  0.05121687]]
Paul Regan Adams (born 20 January 1977) is a former South African cricketer.
[[-0.01312288  0.0900583   0.20340604 ...  0.07938757  0.03795533
   0.07059093]]
Inatio Akaruru CBE (31 March 1937 – 31 January 2014) was a Cook Islands politician, Cabinet Minister and Deputy Prime Minister.
[[0.0753592  0.04943122 0.20363668 ... 0.06575292 0.15284145 0.15586525]]
Eric Cameron Armishaw (1906 – 30 May 1971) was a New Zealand local-body politician and boxing referee.
[[0.01740709 0.09449054 0.12933452 ... 0.07604551 0.06433039 0.09502506]]
Tommy Armstrong Jr. (born November 8, 1993) is an American football quarterback for Sioux Falls Storm of the Indoor Football League (IFL).
[[0.03174324 0.13327655 0.16250138 ... 0.08815113 0.05000255 0.07393596

[[-0.01807702  0.07226838  0.16327362 ...  0.0493938   0.04261024
   0.10719024]]
Colonel Arthur Espie Porritt, Baron Porritt,  (10 August 1900 – 1 January 1994) was a New Zealand physician, military surgeon, statesman and athlete.
[[0.00578941 0.04943122 0.12918545 ... 0.04487365 0.09911086 0.09906054]]
Theodorus Jacobus Leonardus "Dick" Quax (1 January 1948 – 28 May 2018) was a Dutch-born New Zealand runner, one-time world record holder in the 5000 metres, and local-body politician.
[[0.00079854 0.14072578 0.14819935 ... 0.10943598 0.04859019 0.10341408]]
Paul Quinn College (PQC) is a private historically black Methodist college in Dallas, Texas.
[[-0.01312288  0.05339834  0.03124345 ...  0.05308102  0.03369996
   0.06938932]]
William Lee Rees (16 December 1836 – 18 May 1912) was an English-born New Zealand cricketer, politician and lawyer.
[[-0.02874381  0.04943122  0.16267824 ...  0.0058507   0.09047449
   0.08504023]]
Ben Sandford (born 12 March 1979 in Rotorua) is a New Zealand s

[[0.01190357 0.04943122 0.10578334 ... 0.04951847 0.05583615 0.07279273]]
Donald Ray Lash (August 15, 1912 – September 19, 1994) was an American long-distance runner who won 12 national titles from 1934 to 1940, including seven consecutive men's national cross-country championships, and who set a world's record for the two-mile run in 1936.
[[0.00382293 0.1955309  0.15397118 ... 0.05823573 0.09293866 0.10860557]]
Charles Thomas McMillen (born May 26, 1952) is a retired professional basketball player, Rhodes Scholar, and Democratic U.S. Ralph Harold Metcalfe Sr. (May 29, 1910 – October 10, 1978) was an American track and field sprinter and politician.
[[-0.00519743  0.15429512  0.1733776  ...  0.05693707  0.07920743
   0.10360759]]
Larry Jack Mildren (October 10, 1949 – May 22, 2008), was an All-American quarterback at the University of Oklahoma, and professional football player with the Baltimore Colts and New England Patriots.
[[0.00596439 0.10710664 0.22954565 ... 0.04148607 0.088182

[[-0.02874381  0.19396026  0.11650616 ...  0.09768764  0.08189334
   0.09111338]]


In [24]:
#Load test questions dataset
import pandas as pd
df = pd.read_csv('questions.csv')

In [25]:
questions = list(df["question"])

In [26]:
len(questions)

401

In [27]:
for i in range(len(questions)):
    print(i)
    dict_embeddings[questions[i]] = infersent.encode([questions[i]], tokenize=True)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [28]:
#View Sentence Vector for this query
dict_embeddings['Who is the 45th and current president of the United States?'][0]

array([-0.02042911,  0.06372681,  0.10245018, ..., -0.00508549,
        0.01406335,  0.10593148], dtype=float32)

In [29]:
#Create two dictionaries based on even and odd number of entries
d1 = {key:dict_embeddings[key] for i, key in enumerate(dict_embeddings) if i % 2 == 0}
d2 = {key:dict_embeddings[key] for i, key in enumerate(dict_embeddings) if i % 2 == 1}

In [30]:
d1

{'Barack Hussein Obama II ( (listen); born August 4, 1961) is an American politician and attorney who served as the 44th president of the United States from 2009 to 2017.': array([[-0.02874381,  0.04943122,  0.20910737, ...,  0.1316887 ,
          0.04436744,  0.11170954]], dtype=float32),
 'George Walker Bush (born July 6, 1946) is an American politician and businessman who served as the 43rd president of the United States from 2001 to 2009.': array([[-0.02874381,  0.04943122,  0.16229874, ...,  0.06349707,
          0.054673  ,  0.1077443 ]], dtype=float32),
 'George Washington (February 22, 1732 – December 14, 1799) was an American political leader, military general, statesman, and Founding Father who served as the first president of the United States from 1789 to 1797.': array([[-0.02874381,  0.04943122,  0.1495678 , ...,  0.03927369,
          0.13337407,  0.114299  ]], dtype=float32),
 'John Fitzgerald Kennedy (May 29, 1917 – November 22, 1963), often referred to by his initials 

In [31]:
d2

{'Joseph Robinette Biden Jr. (; born November 20, 1942) is an American politician who served as the 47th vice president of the United States from 2009 to 2017 and represented Delaware in the U.S. Senate from 1973 to 2009.': array([[0.01917268, 0.05178139, 0.21973753, ..., 0.06421494, 0.0845053 ,
         0.11733628]], dtype=float32),
 'Donald John Trump (born June 14, 1946) is the 45th and current president of the United States.': array([[-0.01994209,  0.06318004,  0.1411632 , ...,  0.02324948,
          0.08776435,  0.09802058]], dtype=float32),
 'Thomas Jefferson (April 13, 1743 – July 4, 1826) was an American statesman, diplomat, lawyer, architect, philosopher, and Founding Father who served as the third president of the United States from 1801 to 1809.': array([[-0.02874381,  0.04943122,  0.09886527, ...,  0.03141018,
          0.14112839,  0.11247957]], dtype=float32),
 'Theodore  Roosevelt Jr. ( ROH-zə-velt; October 27, 1858 – January 6, 1919) was an American statesman, politicia

In [32]:
#Dump dictionary
with open('data/dict_embeddings1.pickle', 'wb') as handle:
    pickle.dump(d1, handle)

In [33]:
#Dump dictionary
with open('data/dict_embeddings2.pickle', 'wb') as handle:
    pickle.dump(d2, handle)